In [ ]:
USE {
    dependencies {
        implementation("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.3")
    }
}

In [ ]:
import com.fasterxml.jackson.databind.ObjectMapper
import java.io.BufferedReader
import java.io.InputStreamReader
import java.net.HttpURLConnection
import java.net.URL

val domain = "http://localhost:8080"
fun readUrl(urlString: String): String {
    var reader: BufferedReader? = null
    var result = "{}"
    try {
        val url = URL(urlString)
        reader = BufferedReader(InputStreamReader(url.openStream()))
        val buffer = StringBuffer()
        var read: Int
        val chars = CharArray(1024)
        while (reader.read(chars).also { read = it } != -1) buffer.append(chars, 0, read)
        result = buffer.toString()
    } finally {
        reader?.close()
    }
    return result
}
val problemsCount = readUrl("${domain}/problems").count { it == '}' }
val ids = (1..getProblemsCount()).toList().sortedByDescending { abs(it - 18) }

In [34]:
import com.fasterxml.jackson.annotation.JsonProperty
import java.util.stream.Collectors

data class Task(
    @JsonProperty("room_width")
    val room_width: Double,
    @JsonProperty("room_height")
    val room_height: Double,
    @JsonProperty("stage_width")
    val stage_width: Double,
    @JsonProperty("stage_height")
    val stage_height: Double,
    @JsonProperty("stage_bottom_left")
    val stage_bottom_left: List<Double>,
    @JsonProperty("musicians")
    val musicians: List<Int>,
    @JsonProperty("attendees")
    val attendees: List<Attendee>,
    @JsonProperty("pillars")
    val pillars: List<Pillars>
) {
    companion object {
        fun parse(json: String): Task {
            return ObjectMapper().readValue(json, Task::class.java)
        }
    }
}

data class Attendee(
    @JsonProperty("x")
    val x: Double,
    @JsonProperty("y")
    val y: Double,
    @JsonProperty("tastes")
    val tastes: List<Double>
)

data class Pillars(
    @JsonProperty("center")
    val center: List<Double>,
    @JsonProperty("radius")
    val radius: Double
)

data class Solve(
    @JsonProperty("placements")
    val placements: List<Point>,
    @JsonProperty("volumes")
    val volumes: List<Double>? = (1..placements.size).map { 10.0 }
) {
    companion object {
        fun parse(json: String): Solve {
            return ObjectMapper().readValue(json, Solve::class.java)
        }
    }
}

data class Point(
    @JsonProperty("x")
    val x: Double,
    @JsonProperty("y")
    val y: Double
) {
    operator fun plus(p: Point) = Point(x + p.x, y + p.y)

    operator fun minus(p: Point) = Point(x - p.x, y - p.y)

    operator fun times(d: Double) = Point(x * d, y * d)

    operator fun div(d: Double) = Point(x / d, y / d)

    infix fun scalar(p: Point) = p.x * x + p.y * y

    infix fun dist(p: Point) = sqrt((this - p).sqrSize())

    fun sqrSize() = x * x + y * y

    fun norm() = this / sqrt(sqrSize())
}

In [35]:
fun getCells(problem: Task, scale: Boolean = true): List<Point> {
    val cells = (0..(problem.stage_width.toInt() - 20) step 10).flatMap { x ->
        (0..(problem.stage_height.toInt() - 20) step 10).map { y ->
            Point(
                x.toDouble() + problem.stage_bottom_left[0] + 10.0,
                y.toDouble() + problem.stage_bottom_left[1] + 10.0
            )
        }
    }
    if (!scale) {
        return cells
    }
    val coef = getCoef(problem, cells)
    return cells.map {
        val x = if (coef.x.isNaN()) {
            it.x
        } else {
            (((it.x - (problem.stage_bottom_left[0] + 10.0)) * coef.x + (problem.stage_bottom_left[0] + 10.0)) * 1000).roundToLong() / 1000.0
        }
        val y = if (coef.y.isNaN()) {
            it.y
        } else {
            (((it.y - (problem.stage_bottom_left[1] + 10.0)) * coef.y + (problem.stage_bottom_left[1] + 10.0)) * 1000).roundToLong() / 1000.0
        }
        Point(x, y)
    }
}

fun getCoef(problem: Task, cells: List<Point>): Point {
    val coefX =
        (problem.stage_width - 20) / (cells.maxOfOrNull { it.x }!! - cells.minOfOrNull { it.x }!!)
    val coefY =
        (problem.stage_height - 20) / (cells.maxOfOrNull { it.y }!! - cells.minOfOrNull { it.y }!!)
    val x = if (coefX.isNaN()) {
        1.0
    } else {
        coefX
    }
    val y = if (coefY.isNaN()) {
        1.0
    } else {
        coefY
    }
    return Point(x, y)
}

fun List<Point>.filterBorder(problem: Task, dist: Int = 1): List<Point> {
    return filter {
        val a = min(
            abs(it.x - problem.stage_bottom_left[0]),
            abs(problem.stage_bottom_left[0] + problem.stage_width - it.x)
        )
        val b = min(
            abs(it.y - problem.stage_bottom_left[1]),
            abs(problem.stage_bottom_left[1] + problem.stage_height - it.y)
        )
        min(a, b) < (dist + 1) * 10.0
    }
}

fun <T, R> Collection<T>.pmap(mapper: (T) -> R): List<R> {
    return this.stream().parallel().map { mapper(it) }.collect(Collectors.toList())
}


In [41]:
import icfpc.y2023.model.*
import kotlin.math.ceil
import kotlin.math.sqrt

class CalcScoringService {
    companion object {
        val EPS = 1e-9
        val R = 5.0
    }


    fun calcSimpleInfluence(instr: Int, mPos: Point, att: Attendee): Long {
        val attPoint = Point(att.x, att.y)
        val d = (attPoint - mPos).sqrSize()
        return if (d < EPS) {
            0L
        } else {
            ceil(1_000_000.0 * att.tastes[instr] / d).toLong()
        }
    }

    fun calcInfluencesWithoutVolume(problem: Task, placement: List<Point>): List<Long> {
        val influences = problem.musicians.map { 0L }.toMutableList()
        for (att in problem.attendees) {
            val attPoint = Point(att.x, att.y)
            val pillars = problem.pillars.map { Pillar(it.center[0], it.center[1], it.radius) }.toMutableList()
            val musicianInfluencesForAtt = placement
                .mapIndexed { ind, mPos -> Triple(ind, problem.musicians[ind], mPos) }
                .sortedBy { it.third dist attPoint }
                .map { (ind, instr, mPos) ->
                    val d = (attPoint - mPos).sqrSize()
                    if (d < EPS || intersect(pillars, mPos, attPoint)) {
                        Pair(ind, 0L)
                    } else {
                        pillars += Pillar(mPos.x, mPos.y, R)
                        Pair(ind, ceil(1_000_000.0 * att.tastes[instr] / d).toLong())
                    }
                }
                .toList()
            for ((musInd, influence) in musicianInfluencesForAtt) {
                influences[musInd] += influence
            }
        }
        return influences;

    }

    fun calcWithPillars(problem: Task, instr: Int, mPos: Point, pillars: List<Pillar>): Long {
        var result = 0L
        for (att in problem.attendees) {
            val attPoint = Point(att.x, att.y)
            val d = (attPoint - mPos).sqrSize()
            if (d > EPS && !intersect(pillars, mPos, attPoint)) {
                result += ceil(1_000_000.0 * att.tastes[instr] / d).toLong()
            }
        }
        return result
    }

    fun calc(problem: Task, solve: Solve): Long {
        val list = solve.placements.mapIndexed { i, p -> i to p }
        if (list.any { (ia, a) -> list.any { (ib, b) -> ia != ib && a dist b < 2 * R } }) {
            println("intersect")
            return Long.MIN_VALUE
        }
        if (problem.stage_bottom_left[0] + 10.0 > solve.placements.minOfOrNull { it.x }!! ||
            problem.stage_bottom_left[1] + 10.0 > solve.placements.minOfOrNull { it.y }!! ||
            problem.stage_bottom_left[0] + problem.stage_width - 10.0 < solve.placements.maxOfOrNull { it.x }!! ||
            problem.stage_bottom_left[1] + problem.stage_height - 10.0 < solve.placements.maxOfOrNull { it.y }!!
        ) {
            println("stage")
            return Long.MIN_VALUE
        }

        val influences = calcInfluencesWithoutVolume(problem, solve.placements)

        val togetherCoeffs = solve.placements.mapIndexed { i, p ->
            1.0 + solve.placements.filterIndexed { j, _ -> i != j }
                .sumOf { t -> 1 / (p dist t) }
        }

        return influences
            .zip(solve.volumes ?: influences.map { 1.0 })
            .zip(togetherCoeffs) { (a, b), c -> Triple(a, b, c) }
            .sumOf { (inf, vol, q) -> ceil(inf * q * vol).toLong() }
    }

    fun intersect(musicians: List<Pillar>, a: Point, b: Point): Boolean {
        return musicians.any { intersect(a, b, it) }
    }

    fun intersect(p1: Point, p2: Point, p: Pillar): Boolean {
        val m = Point(p.x, p.y)
        val r = p.r
        val n = p2 - p1
        val mp = p1 + n * (n scalar (m - p1)) / n.sqrSize()
        if ((m dist mp) > r - EPS) return false
        if ((p1 dist mp) + (p2 dist mp) < (p1 dist p2) + EPS) return true
        if (p1 dist m < r + EPS || p2 dist m < r + EPS) return true
        return false
    }

    fun dist(a: Point, b: Point): Double {
        return sqrt((a.x - b.x) * (a.x - b.x) + (a.y - b.y) * (a.y - b.y))
    }

    fun dist(a: Attendee, b: Point): Double {
        return sqrt((a.x - b.x) * (a.x - b.x) + (a.y - b.y) * (a.y - b.y))
    }

    fun inBorders(problem: Task, pos: Point): Boolean {
        return !(problem.stage_bottom_left[0] + 10.0 > pos.x ||
                problem.stage_bottom_left[1] + 10.0 > pos.y ||
                problem.stage_bottom_left[0] + problem.stage_width - 10.0 < pos.x ||
                problem.stage_bottom_left[1] + problem.stage_height - 10.0 < pos.y)
    }
}

val scoring = CalcScoringService()

In [42]:
import java.util.*

fun getPlaceFor(problem: Task): Solve {
    val sourceCells = getCells(problem).shuffled()
    var dist = 1
    val cells = sourceCells.filterBorder(problem, dist = dist).toMutableSet()
    println("cellssize ${cells.size}")
    println("cells ${cells}")
    val mPoints = mutableSetOf<Point>()
    for (m in problem.musicians) {
        if (cells.isEmpty()) {
            dist++
            cells += sourceCells.filterBorder(problem, dist = dist)
                .filter { it !in mPoints }
        }
        val cell = cells.maxBy { mPos ->
            problem.attendees.map({ scoring.calcSimpleInfluence(m, mPos, it) }).sum()
        }
        println("found best place for $m on $cell")
        cells -= cell
        mPoints += cell
    }
    val placement = mPoints.toList()
    return addVolume(problem, placement)
}

fun addVolume(problem: Task, placement: List<Point>): Solve {
    val influenceCoeff = scoring.calcInfluencesWithoutVolume(problem, placement)
        .map { if (it > 0) 10.0 else 0.0 }
    return Solve(placement, influenceCoeff)
}

Line_49.jupyter.kts (29:31 - 40) Type mismatch: inferred type is kotlin.collections.List<Line_41_jupyter.Point> but java.util.List<Line_41_jupyter.Point> was expected
Line_49.jupyter.kts (32:41 - 52) This class shouldn't be used in Kotlin. Use kotlin.collections.List or kotlin.collections.MutableList instead.
Line_49.jupyter.kts (33:71 - 80) Type mismatch: inferred type is java.util.List<Line_41_jupyter.Point> but kotlin.collections.List<Line_41_jupyter.Point> was expected
Line_49.jupyter.kts (35:18 - 27) Type mismatch: inferred type is java.util.List<Line_41_jupyter.Point> but kotlin.collections.List<Line_41_jupyter.Point> was expected

In [ ]:


fun test2(id: Int) {
    val problem = Task.parse(readUrl("$domain/problem/$id"))
    val solution = getPlaceFor(problem)
    val score = scoring.calc(problem, solution)
    URL("$domain/add/$id?calc=true").send(solution)
    println("score for $id: $score")
}